### Variable

In [ ]:
text_path = "/content/drive/MyDrive/Colab Notebooks/milk"
openApiKey = "BD099954D410A11253DA35DEB404198B"
conv_length = 6
quiz_threshold = 5000

### Enviornment Setting

In [ ]:
pip install fasttext

     |████████████████████████████████| 68 kB 4.0 MB/s 
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3126399 sha256=d20daf56aae1cf915ba13d5adb3babbd964754fc4a5a6deb369a838aa79159c0
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
!pip install konlpy
%env JAVA_HOME "C:\Program Files\Java\jdk-14.0.2"

     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 86 kB 4.2 MB/s 
     |████████████████████████████████| 448 kB 16.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
env: JAVA_HOME="C:\Program Files\Java\jdk-14.0.2"


In [ ]:
import fasttext
import konlpy
from konlpy.tag import Okt
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
import json
import os
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import random

In [ ]:
import fasttext.util
fasttext.util.download_model('ko', if_exists='ignore')

'cc.ko.300.bin'

In [ ]:
model = fasttext.load_model('cc.ko.300.bin')

In [ ]:
komoran = Komoran()

In [ ]:
re = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")

### Util

In [ ]:
def search_word(word):
    ret_json = {"word":word, "items":[]}
    
    ## 명사 혹은 대명사면 그대로
    if komoran.pos(word)[0][1] == 'NN' or 'NP':
        word = komoran.pos(word)[0][0]

    params = f"?certkey_no=3114&key={openApiKey}&target_type=search&q={word}"
    openUrl = "https://opendict.korean.go.kr/api/search"+params

    paramsForExample = f"?certkey_no=3114&key={openApiKey}&target_type=search&q={word}&part=exam"
    openUrlForExample = "https://opendict.korean.go.kr/api/search"+paramsForExample

    res = requests.get(openUrl)
    soup = BeautifulSoup(res.content, 'html.parser')

    resEx = requests.get(openUrlForExample)
    soupEx = BeautifulSoup(resEx.content, 'html.parser')

    check = soup.find('total').get_text()
    if check == '0':
        return ret_json

    wordPos = soup.find_all('pos')
    wordDef = soup.find_all('definition')
    wordEx = soupEx.find_all('example')

    for i in range(min(3, len(wordPos))):
        item = {}
        item["pos"] = wordPos[i].get_text().strip()
        item["definition"] = wordDef[i].get_text().strip()
        ret_json["items"].append(item)

    return ret_json

In [ ]:
def cos(A, B):
    return (A.dot(B.T)/(np.linalg.norm(A) * np.linalg.norm(B)))

In [ ]:
def create_answer(problem):
    sentence_vector = model.get_sentence_vector(problem)
    result = []
    problem = problem.replace("?", " ")
    problem = problem.replace(".", " ")
    problem = problem.replace("!", " ")
    problem = problem.replace("~", " ")
    
    for word in problem.split(" "):
        word_vector = model.get_word_vector(word)
        result.append([cos(sentence_vector, word_vector), word])

    result = sorted(result, key=lambda x: -x[0])
    answer = result[0][1]
    return answer

In [ ]:
def create_candidate(answer):
    answer_morphs = komoran.morphs(answer)
    candidate = model.get_nearest_neighbors(answer)
    
    candidate = map(lambda x : x[1], candidate)
    candidate = filter(lambda x : re.search(x) == None, candidate)
    candidate = filter(lambda x : komoran.morphs(x) != answer_morphs, candidate)
    candidate = filter(lambda x : 3 * len(answer) > len(x), candidate )
    candidate = list(candidate)


    if len(candidate) < 3:
        return None
    else:
        return candidate[:3]


In [ ]:
def change_speaker(context):
    speaker =['승열', '돌맹', '붕어', '감자']
    random.shuffle(speaker)
    s = []

    for c in context:
        if c["speaker"] not in s:
            s.append(c["speaker"])
        c["speaker"] = speaker[s.index(c["speaker"])]
    
    return context

## Quiz generator

In [ ]:
def quiz_generator(path):
    with open(path) as dataset:
        dataset = json.load(dataset)['sentences']

    for index in range(0, len(dataset), conv_length):
##    for index in range(0, 3, conv_length):
        global quiz_num

        if index + conv_length > len(dataset):
            break

        ## init
        problem_commit = {
            "context": [],
            "option": [],
            "solution":0
        }
        problem = ""
        context = []
        option = []

        ## problem
        for conv_index in range(conv_length):
            problem += dataset[index + conv_index]['content']
            context.append({
                "content": dataset[index + conv_index]['content'],
                "speaker": dataset[index + conv_index]['speaker']
            })
        context = change_speaker(context)
        
        ## answer
        answer = create_answer(problem)
        candidate = create_candidate(answer)

        if candidate == None:
            continue
        
        candidate.append(answer)
        random.shuffle(candidate)

        ## answer sheet
        for c in candidate:
            option.append(search_word(c))


        problem_commit["context"] = context
        problem_commit["option"] = option
        problem_commit["solution"] = candidate.index(answer)

        ## write in json
        with open("{0}/output3/{1}.json".format(text_path, quiz_num), 'w') as f:
            json.dump(problem_commit, f, ensure_ascii=False)
        

        
        quiz_num += 1
        

## Main

In [ ]:
quiz_num = 0

for file_name in os.listdir(text_path):
    print(file_name)
    if quiz_num > quiz_threshold:
        break
    quiz_generator(text_path + '/' +file_name)


out_SDRW2000001158.json


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in float_scalars
  


KeyboardInterrupt: ignored